In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = "glm-4-flash",
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
    openai_api_key = "",
)

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun

In [12]:
import pandas as pd
from pandasql import sqldf
def simulate_database_operation(sql):
    my_table = pd.DataFrame({
    'name':['Henry Myers', 'Martha Hawkins','Kelsey Lutz','Jonathan Fowler','Jonathan Young', "Kimberly Macias", "7", "8", "9", "10"],
    'age':[68, 44, 54, 46, 76, 22, 69, 33, 23, 351],
    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M'],
    })
    result = sqldf(sql)
    return result
print(simulate_database_operation('SELECT * FROM my_table WHERE age > 58'))

             name  age sex
0     Henry Myers   68   F
1  Jonathan Young   76   F
2               7   69   M
3              10  351   M


In [13]:
from langchain.tools import tool

@tool
def simulate_database_operation(sql):
    '''根据sql语句操作数据库中的my_table表'''
    my_table = pd.DataFrame({
    'name':['Henry Myers', 'Martha Hawkins','Kelsey Lutz','Jonathan Fowler','Jonathan Young', "Kimberly Macias", "7", "8", "9", "10"],
    'age':[68, 44, 54, 46, 76, 22, 69, 33, 23, 351],
    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M'],
    })
    result = sqldf(sql)
    return result

In [14]:
tools = [DuckDuckGoSearchRun(), simulate_database_operation]
model_with_tools = model.bind_tools(tools)

In [16]:
from pprint import pp

response = model_with_tools.invoke("地球上有多少个国家？")
pp(dict(response))

{'content': '截至2023年，世界上共有195个主权国家，以及2个观察员国家。这些国家被国际社会普遍承认，并在联合国拥有正式成员资格。这两个观察员国家是梵蒂冈（教廷）和巴勒斯坦。需要注意的是，这个数字可能会随着国际政治形势的变化而变化。',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 66,
                                       'prompt_tokens': 270,
                                       'total_tokens': 336,
                                       'completion_tokens_details': None,
                                       'prompt_tokens_details': None},
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'id': '202504132345524606d06db50949a7',
                       'finish_reason': 'stop',
                       'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-5681de46-bfd0-4d23-a3ae-450bcb2ef6fc-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 270,
                    'output_tokens': 66,
                

In [17]:
response = model_with_tools.invoke("深圳今天天气怎么样？")
pp(dict(response))

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_-8802638241889870695',
                                       'function': {'arguments': '{"query": '
                                                                 '"深圳今天天气"}',
                                                    'name': 'duckduckgo_search'},
                                       'type': 'function',
                                       'index': 0}],
                       'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 28,
                                       'prompt_tokens': 270,
                                       'total_tokens': 298,
                                       'completion_tokens_details': None,
                                       'prompt_tokens_details': None},
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'id': '20250413234658ef9c1f61e2f84e99',
                       

In [18]:
db_response = model_with_tools.invoke("往数据库中添加一条数据，name是张三，age是18，sex是male")
pp(dict(db_response))

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_-8802647519020908762',
                                       'function': {'arguments': '{"sql": '
                                                                 '"INSERT INTO '
                                                                 'my_table '
                                                                 '(name, age, '
                                                                 'sex) VALUES '
                                                                 "('张三', 18, "
                                                                 '\'male\')"}',
                                                    'name': 'simulate_database_operation'},
                                       'type': 'function',
                                       'index': 0}],
                       'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 32,
                                       'prompt_t

In [ ]:
def manual_agent(query: str, model: ChatOpenAI, tools: list[tool]):
    model_with_tools = model.bind_tools(tools)
    response = model_with_tools.invoke(query)
    tool_response = call_tool(response, tools)
    final_response = model.invoke(
        f"original_query: {query}\n\n tools_response: {tool_response}"
    )
    return final_response


def call_tool(model_output, tools):
    tools_map = {tool.name.lower():tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool['name']
        tool_args = tool['args']
        tool_instance = tools_map[tool_name]
        tool_response = tool_instance.invoke(*tool_args.values())
        tools_response[tool_name] = tool_response
    return tools_response

manual_agent("查询my_tabel表中年龄大于60岁的人名", model, tools).content

'根据提供的查询和工具响应，以下是对查询结果的解释：\n\n查询要求从名为 `my_table` 的表中查询年龄大于60岁的人名。\n\n工具响应显示了一个结果列表，其中包含以下内容：\n- 0: Henry Myers\n- 1: Jonathan Young\n- 2: 7\n- 3: 10\n\n从工具响应中可以推断出以下几点：\n1. 有两个人的名字被正确返回：Henry Myers 和 Jonathan Young。\n2. 接下来的两个条目（7 和 10）可能是不相关的数据，或者可能是查询结果中的一个错误，因为它们没有名字。\n\n基于上述信息，可以得出以下结论：\n\n查询结果中，年龄大于60岁的两个人名是 Henry Myers 和 Jonathan Young。其他数据可能是查询过程中的一个错误或者不相关数据。'

In [22]:
'''处理agent复杂调用逻辑'''
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model, tools)

In [24]:
from langchain_core.messages import HumanMessage

response = agent.invoke(
    {'messages':[HumanMessage('帮我查询数据库my_table表中有多少人年龄大于60')]}
)
response

{'messages': [HumanMessage(content='帮我查询数据库my_table表中有多少人年龄大于60', additional_kwargs={}, response_metadata={}, id='f6eb3fb7-796d-4426-bcd4-bafe6cffb5a5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_-8802645010759408948', 'function': {'arguments': '{"sql": "SELECT COUNT(*) FROM my_table WHERE age > 60"}', 'name': 'simulate_database_operation'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 276, 'total_tokens': 297, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'id': '20250414002928056ccc7978d74fee', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cb182eba-4cc9-4011-884c-08723eb8700e-0', tool_calls=[{'name': 'simulate_database_operation', 'args': {'sql': 'SELECT COUNT(*) FROM my_table WHERE age > 60'}, 'id': 'call_-8802645010759408948', 'type': 'tool_call'}], usage_metadata={'input_tokens': 27

In [25]:
response = agent.invoke(
    {'messages':[HumanMessage('今天深圳的天气怎么样？')]}
)
response

/Users/chenjianxu/anaconda3/envs/agent_demo/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


{'messages': [HumanMessage(content='今天深圳的天气怎么样？', additional_kwargs={}, response_metadata={}, id='75d7b58e-54c5-420f-8222-8e00540bf14d'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_-8802643052254199419', 'function': {'arguments': '{"query": "深圳今天天气"}', 'name': 'duckduckgo_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 271, 'total_tokens': 299, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'id': '20250414003021ee89c13634144c71', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bddb4d8e-d713-4314-8d30-3ee1b7a77a66-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '深圳今天天气'}, 'id': 'call_-8802643052254199419', 'type': 'tool_call'}], usage_metadata={'input_tokens': 271, 'output_tokens': 28, 'total_tokens': 299, 'input_token_details': {}, 'output_token_details': {}}),
  Tool